# 1 Pengolahan Hasil Face Landmark (untuk Analisis / Training)

🎯 Tujuan:
1. Mendapatkan data (x, y, z) dari landmark wajah

2. Bisa dipakai untuk klasifikasi, analisis ekspresi, atau training ML


🎯 Setelah dapat landmark dari Face Mesh atau Face Landmarker, kita bisa:
* Ambil koordinat (x, y, z) dari tiap titik wajah

* Simpan dalam array atau dataframe

* Normalize data (misalnya terhadap bounding box wajah)

* Simpan ke file .csv buat training model ML



In [1]:
import cv2
import mediapipe as mp

In [ ]:
landmark_list = []
for lm in face_landmarks.landmark:
    landmark_list.extend([lm.x, lm.y, lm.z])

# lm.x, lm.y, lm.z adalah koordinat relatif (dalam skala 0–1)
# Jumlah total titik: 468 landmark (default Face Mesh)
# Bisa dikurangi ke area tertentu (misal: mata, bibir, dll)

In [ ]:
import numpy as np
coords = np.array([[lm.x, lm.y, lm.z] for lm in face_landmarks.landmark])

In [ ]:
# Misal normalisasi terhadap titik tengah wajah (titik ke-1)
ref_x = face_landmarks.landmark[1].x
ref_y = face_landmarks.landmark[1].y

normalized = []
for lm in face_landmarks.landmark:
    normalized.append([lm.x - ref_x, lm.y - ref_y])


# 2 Ekstraksi ke CSV dari Webcam atau Video

🎯 Tujuan:
* Menyimpan hasil koordinat landmark (dan label, kalau ada)

* Bisa digunakan untuk machine learning, data analisis, atau pembuatan dataset gesture/ekspresi

🎯 Kita bisa bikin skrip untuk:

* Baca webcam / video

* Proses wajah per frame

* Simpan data landmark + label (jika ada)

In [ ]:
# Setiap baris = 1 frame
# Setiap kolom = koordinat landmark (x1, y1, z1, x2, y2, z2, ..., label)

In [ ]:
import cv2
import mediapipe as mp
import csv

# Inisialisasi MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Buka webcam / video
cap = cv2.VideoCapture(0)  # ganti dengan path video kalau pakai dataset

# Siapkan file CSV
csv_file = open('landmark_output_netral.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)

# Tulis header CSV (x1, y1, z1, ..., label)
header = []
for i in range(468):  # atau ganti jumlah titik jika dibatasi
    header += [f'x{i}', f'y{i}', f'z{i}']
header.append('label')  # kalau pakai label gesture / ekspresi
csv_writer.writerow(header)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Proses frame
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            row = []
            for lm in face_landmarks.landmark:
                row += [lm.x, lm.y, lm.z]

            label = 'neutral'  # bisa disesuaikan manual atau dari sistem
            row.append(label)
            csv_writer.writerow(row)

    # Tampilkan (opsional)
    cv2.imshow('Recording Face Landmark', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Tutup semua
csv_file.close()
cap.release()
cv2.destroyAllWindows()


# 3 Implementasi MediaPipe ke Dataset Offline (Video/Image)

🎯 Tujuan:
* Menggunakan MediaPipe bukan dari webcam langsung, tapi dari file video atau folder gambar sebagai dataset.

🎯 Ada 2 jenis dataset offline:
1. Dataset Berupa File Video (.mp4, .avi, dsb)
2. Dataset Berupa Gambar (folder berisi .jpg, .png, dll)

In [2]:
# Proses Dataset Berupa Video

import cv2
import mediapipe as mp
import csv

# Inisialisasi MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, refine_landmarks=True)

# Buka file video
video_path = "dataset/video/Abuse006_x264_21.mp4"
cap = cv2.VideoCapture(video_path)

# Siapkan file CSV
csv_file = open('video_landmark_output_video.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)

# Tulis header
header = []
for i in range(468):
    header += [f'x{i}', f'y{i}', f'z{i}']
header.append('label')
csv_writer.writerow(header)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            row = []
            for lm in face_landmarks.landmark:
                row += [lm.x, lm.y, lm.z]
            row.append('senyum')  # contoh label manual dari nama video
            csv_writer.writerow(row)

cap.release()
csv_file.close()


I0000 00:00:1744855057.073342    7995 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1744855057.080896    8046 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.2.5), renderer: STONEY (stoney, LLVM 15.0.6, DRM 3.42, 5.15.0-76-generic)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1744855057.142163    8039 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744855057.196077    8040 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744855058.375120    8039 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [ ]:
# Proses Dataset Berupa Gambar dalam Folder

import cv2
import mediapipe as mp
import os
import csv

# Inisialisasi Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)

# Folder gambar
image_folder = "dataset/gambar/"
image_list = os.listdir(image_folder)

# Siapkan CSV
csv_file = open('gambar_landmark_output_gambar.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)

# Tulis header
header = []
for i in range(468):
    header += [f'x{i}', f'y{i}', f'z{i}']
header.append('label')
csv_writer.writerow(header)

for img_name in image_list:
    path = os.path.join(image_folder, img_name)
    img = cv2.imread(path)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            row = []
            for lm in face_landmarks.landmark:
                row += [lm.x, lm.y, lm.z]
            # contoh: ambil label dari nama file, misalnya senyum_1.jpg
            label = img_name.split('_')[0]
            row.append(label)
            csv_writer.writerow(row)

csv_file.close()


# 4 Gesture Detection / Face Analysis
🎯 Tujuan:
* Menganalisis ekspresi atau gesture wajah (misalnya senyum, kedipan, buka mulut) berdasarkan koordinat landmark dari MediaPipe.

**Konsep Umum: Analisis Berdasarkan Jarak / Rasio**

🎯 Contoh-contoh gesture atau ekspresi:
* Senyum → bibir melebar

* Mata tertutup → tinggi mata menurun

* Mulut terbuka → jarak atas-bawah bibir besar

* Mengangguk / menggeleng → arah kepala berubah

In [ ]:
# Deteksi Mulut Terbuka
import math

def euclidean_distance(p1, p2):
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

# Misalnya deteksi dari FaceMesh
top_lip = face_landmarks.landmark[13]
bottom_lip = face_landmarks.landmark[14]

# Hitung jarak antara bibir atas & bawah
mouth_distance = euclidean_distance(top_lip, bottom_lip)

if mouth_distance > 0.05:
    print("Mulut terbuka")


In [ ]:
# Deteksi Kedipan Mata (Eye Aspect Ratio)
# Titik sekitar mata kiri (misalnya)
left_eye_top = face_landmarks.landmark[386]
left_eye_bottom = face_landmarks.landmark[374]

eye_distance = euclidean_distance(left_eye_top, left_eye_bottom)

if eye_distance < 0.01:
    print("Mata kiri terpejam")


In [ ]:
# Deteksi Senyum
left_lip = face_landmarks.landmark[61]
right_lip = face_landmarks.landmark[291]

lip_width = euclidean_distance(left_lip, right_lip)

if lip_width > 0.3:
    print("Senyum")


In [ ]:
import cv2
import mediapipe as mp
import math

# Fungsi untuk hitung jarak
def distance(p1, p2):
    return ((p1.x - p2.x)**2 + (p1.y - p2.y)**2) ** 0.5

# Inisialisasi Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Buka Webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    label = ""

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # Ambil titik-titik bibir untuk analisis
            left_lip = landmarks.landmark[61]
            right_lip = landmarks.landmark[291]
            top_lip = landmarks.landmark[13]
            bottom_lip = landmarks.landmark[14]

            # Jarak horizontal dan vertikal bibir
            lip_width = distance(left_lip, right_lip)
            mouth_open = distance(top_lip, bottom_lip)

            # Threshold sederhana (bisa di-tune)
            label_list = []
            if mouth_open > 0.05:
                label_list.append("Mulut Terbuka")
            if lip_width > 0.15:
                label_list.append("Senyum")

            label = " & ".join(label_list) if label_list else "Netral"

            # Gambar landmark wajah
            mp_drawing.draw_landmarks(
                frame,
                landmarks,
                mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,128,255), thickness=1)
            )

    # Tampilkan label prediksi
    cv2.putText(frame, f'Prediksi: {label}', (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Tampilkan frame
    cv2.imshow("Deteksi Ekspresi Real-time", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Mulut Membuka Tertutup & Mata Terbuka Tertutup

import cv2
import mediapipe as mp
import math

# Fungsi untuk hitung jarak Euclidean
def distance(p1, p2):
    return ((p1.x - p2.x)**2 + (p1.y - p2.y)**2) ** 0.5

# Fungsi untuk menghitung Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks, landmarks):
    p1 = landmarks[eye_landmarks[0]]  # Titik horizontal kiri
    p2 = landmarks[eye_landmarks[1]]  # Titik vertikal atas 1
    p3 = landmarks[eye_landmarks[2]]  # Titik vertikal atas 2
    p4 = landmarks[eye_landmarks[3]]  # Titik horizontal kanan
    p5 = landmarks[eye_landmarks[4]]  # Titik vertikal bawah 1
    p6 = landmarks[eye_landmarks[5]]  # Titik vertikal bawah 2

    vertical_1 = distance(p2, p6)
    vertical_2 = distance(p3, p5)
    horizontal = distance(p1, p4)

    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Inisialisasi Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Indeks landmark untuk mata kiri dan kanan
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]

# Buka Webcam
cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    label = ""

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # Hitung EAR untuk mata kiri dan kanan
            left_ear = calculate_ear(LEFT_EYE, landmarks.landmark)
            right_ear = calculate_ear(RIGHT_EYE, landmarks.landmark)
            avg_ear = (left_ear + right_ear) / 2.0

            # Deteksi mata terbuka atau tertutup
            eye_status = "Mata Terbuka" if avg_ear >= 0.2 else "Mata Tertutup"

            # Ambil titik-titik bibir untuk analisis mulut
            top_lip = landmarks.landmark[13]
            bottom_lip = landmarks.landmark[14]

            # Jarak vertikal bibir
            mouth_open = distance(top_lip, bottom_lip)

            # Deteksi mulut terbuka atau tertutup
            mouth_status = "Mulut Terbuka" if mouth_open > 0.05 else "Mulut Tertutup"

            # Gabungkan label
            label = f"{eye_status} & {mouth_status}"

            # Gambar landmark wajah
            mp_drawing.draw_landmarks(
                frame,
                landmarks,
                mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=1)
            )

    # Tampilkan label prediksi
    cv2.putText(frame, f'Prediksi: {label}', (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Tampilkan frame
    cv2.imshow("Deteksi Ekspresi Real-time", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1744857172.259086    7995 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1744857172.272582    9054 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.2.5), renderer: STONEY (stoney, LLVM 15.0.6, DRM 3.42, 5.15.0-76-generic)
W0000 00:00:1744857172.283156    9052 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744857172.347347    9051 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


KeyboardInterrupt: 

: 